In [46]:
import os
from dotenv import load_dotenv
import requests
import base64
import json

# Load environment variables from .env file
load_dotenv()

# Get the environment variables
CLIENT_ID = os.getenv('EBAY_APP_ID')
CLIENT_SECRET = os.getenv('EBAY_CERT_ID')
OAUTH_URL = 'https://api.ebay.com/identity/v1/oauth2/token'

# Encode the client ID and client secret
credentials = base64.b64encode(f'{CLIENT_ID}:{CLIENT_SECRET}'.encode('utf-8')).decode('utf-8')

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': f'Basic {credentials}'
}

data = {
    'grant_type': 'client_credentials',
    'scope': 'https://api.ebay.com/oauth/api_scope'
}

response = requests.post(OAUTH_URL, headers=headers, data=data)
if response.status_code == 200:
    access_token = response.json()['access_token']
    # print(f'Access Token: {access_token}')
else:
    print(f'Error: {response.status_code}')
    print(response.json())

In [59]:
import requests
import json

def search_ebay_items(params, access_token=access_token):
    endpoint = 'https://api.ebay.com/buy/browse/v1/item_summary/search'
    
    # Set up the request headers
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    
    # Make the API request
    response = requests.get(endpoint, headers=headers, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        items = data.get('itemSummaries', [])

        q_param = params.get('q', 'results').replace(' ', '_')
        file_name = f'data_ebay/ebay_browse_search_results_2.json'
        
        # Print and save the response_2
        with open(file_name, 'w') as file:
            json.dump(items, file, indent=4)
        print(items)
    else:
        print(f'Error: {response.status_code}')
        print(response.json())

def get_category_id_by_name(category_name, access_token=access_token):
    # Define the endpoint and headers
    url = 'https://api.ebay.com/commerce/taxonomy/v1/category_tree/0/get_category_suggestions'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    
    # Define the query parameters
    params = {
        'q': category_name  # The name of the category you want to search for
    }
    
    # Make the GET request
    response = requests.get(url, headers=headers, params=params)
    
    # Check the response status and parse the response
    if response.status_code == 200:
        data = response.json()
        # formatted_json = json.dumps(data, indent=4)
        # print(formatted_json)
        categories = data.get('categorySuggestions')
        
        # Print out category names and IDs
        for category in categories:
            print(f"Category Name: {category['category']['categoryName']}, Category ID: {category['category']['categoryId']}")
        return categories
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [49]:
category_name = 'health_beauty'  # Replace with the desired category name
x = get_category_id_by_name(category_name, access_token)
category_id = x[0].get('category').get('categoryId')

Category Name: Anti-Aging Products, Category ID: 33164
Category Name: CCG Individual Cards, Category ID: 183454
Category Name: Vitamins & Minerals, Category ID: 11776
Category Name: Books, Category ID: 261186
Category Name: Women's Fragrances, Category ID: 11848
Category Name: Wheelchairs, Category ID: 19265
Category Name: Moisturizers, Category ID: 21205
Category Name: Eye Shadow, Category ID: 36865
Category Name: Magazines, Category ID: 280


In [40]:
params = {
    'q': '',  # The search query
    'category_id': category_id,
    'limit': 40
}

search_ebay_items(params)

40


In [63]:
params = {
    'q': 'health_beauty',  # Search query for 'drone'
    'limit': '5',  # Limit to 5 results
    'filter': 'price:[5..10],priceCurrency:USD',  # Filters for price range, currency, and condition
}

search_ebay_items(params)

[{'itemId': 'v1|235082658076|0', 'title': 'Royal Jelly & Vitamin E Skin Oil 90 Capsule', 'leafCategoryIds': ['33164'], 'categories': [{'categoryId': '33164', 'categoryName': 'Anti-Aging Products'}, {'categoryId': '26395', 'categoryName': 'Health & Beauty'}, {'categoryId': '11863', 'categoryName': 'Skin Care'}], 'image': {'imageUrl': 'https://i.ebayimg.com/images/g/UEgAAOSw25pmo-sZ/s-l225.jpg'}, 'price': {'value': '7.77', 'currency': 'USD'}, 'itemHref': 'https://api.ebay.com/buy/browse/v1/item/v1%7C235082658076%7C0', 'seller': {'username': 'dailyvita', 'feedbackPercentage': '99.3', 'feedbackScore': 251380}, 'marketingPrice': {'originalPrice': {'value': '15.14', 'currency': 'USD'}, 'discountPercentage': '49', 'discountAmount': {'value': '7.37', 'currency': 'USD'}}, 'condition': 'New', 'conditionId': '1000', 'thumbnailImages': [{'imageUrl': 'https://i.ebayimg.com/images/g/UEgAAOSw25pmo-sZ/s-l1600.jpg'}], 'shippingOptions': [{'shippingCostType': 'FIXED', 'shippingCost': {'value': '0.00', '

In [31]:
category_name = 'health_beauty'  # Replace with the desired category name
x = get_category_id_by_name(category_name, access_token)

{
    "categorySuggestions": [
        {
            "category": {
                "categoryId": "33164",
                "categoryName": "Anti-Aging Products"
            },
            "categoryTreeNodeLevel": 3,
            "categoryTreeNodeAncestors": [
                {
                    "categoryId": "11863",
                    "categoryName": "Skin Care",
                    "categoryTreeNodeLevel": 2,
                    "categorySubtreeNodeHref": "https://api.ebay.com/commerce/taxonomy/v1/category_tree/0/get_category_subtree?category_id=11863"
                },
                {
                    "categoryId": "26395",
                    "categoryName": "Health & Beauty",
                    "categoryTreeNodeLevel": 1,
                    "categorySubtreeNodeHref": "https://api.ebay.com/commerce/taxonomy/v1/category_tree/0/get_category_subtree?category_id=26395"
                }
            ]
        },
        {
            "category": {
                "categoryId": 

In [28]:
type(x[0])

dict